In [5]:
import pandas as pd 
import numpy as np
import datetime

pd.set_option('display.max_columns', None)

#load data
df = pd.read_csv('../../data/127000_rand.csv')

## Functions

In [6]:
def load_data():
    """
    loads sample data extracted from db
    """
    return pd.read_csv('../../data/large_rand.csv') #change this to the path of file

######Inital Processing######
def remove_rows(df):
    """
    remove outlier rows and certain null values
        - arr_delay outliers above 350
        - arr_delay where null
    """

    df = df.loc[(df['arr_delay'].notnull()) | (df['arr_delay'] <= 350)]
    df = df.loc[df['arr_delay'] <= 350]
    #df = df.loc[df['taxi_out'] <= 75]
    #df = df.loc[df['taxi_in'] <= 75]

    return df

def create_datetime(df):
    """
    creates columns for month, year, hour onto the dataframe
    """
    df['month'] = pd.DatetimeIndex(df['fl_date']).month
    df['year'] = pd.DatetimeIndex(df['fl_date']).year
    df['day'] = pd.DatetimeIndex(df['fl_date']).dayofweek
    df['dep_hour'] = df['crs_dep_time'].round(-2)/100
    df['arr_hour']= df['crs_arr_time'].round(-2)/100

    return df

def create_is_late(df):
    """
    creates a column 0/1 to indicate if the flight was late or not
    """
    df['is_late'] = df['arr_delay'].apply(lambda x: 1 if x > 0 else 0)

    return df

def create_speed(df):
    """
    create column calculating expected speed of flight. miles/minute
    """
    df['speed'] = (df['distance'] / df['crs_elapsed_time']).round(2)

    return df

 

######Table Creations######

    ## Airports Table ##
def create_table_airports(df=df):
    """
    input flights table and create a unique airports table with stats that will be used for features later
    """
    df_airports = df.copy()
    df_airports = df[['origin_airport_id', 'origin', 'origin_city_name']]
    df_airports = df_airports.drop_duplicates(subset=['origin_airport_id'])
    df_airports.rename(columns={'origin_airport_id': 'airport_id', 'origin':'airport_code'}, inplace=True)
    split = df_airports['origin_city_name'].str.split(",", n=1, expand=True)
    df_airports['city'] = split[0]
    df_airports['state'] = split[1]
    df_airports.drop('origin_city_name', axis=1, inplace=True)
    
    return df_airports

def create_dep_delay(df_flights, df_airports):
    """
    creates a column showing mean departure delay for each airport
    we used mean because previously have 0d out any negatives and mean results in 0 99% of the time
    """
    airports_temp = pd.DataFrame(df_flights.groupby('origin_airport_id').agg('mean')['dep_delay'])
    airports_temp['dep_delay'] = airports_temp['dep_delay'].round(2)
    airports_temp.reset_index(inplace=True)
    airports_temp.rename(columns={'origin_airport_id':'airport_id'}, inplace=True)

    df_airports = df_airports.merge(airports_temp, on='airport_id')
    df_airports.rename(columns={'dep_delay':'mean_d_delay'}, inplace=True)


    return df_airports

def create_arr_delay(df_flights, df_airports):
    """
    creates a column showing mean arrival delay for each airport
    we used mean because previously have 0d out any negatives and medium results in 0 99% of the time
    """
    airports_temp = pd.DataFrame(df_flights.groupby('dest_airport_id').agg('mean')['arr_delay'])
    airports_temp['arr_delay'] = airports_temp['arr_delay'].round(2)
    airports_temp.reset_index(inplace=True)
    airports_temp.rename(columns={'dest_airport_id':'airport_id'}, inplace=True)

    df_airports = df_airports.merge(airports_temp, on='airport_id')
    df_airports.rename(columns={'arr_delay':'mean_arr_delay'}, inplace=True)

    return df_airports

def create_flight_counts(df_flights, df_airports):
    """
    add columns counting amount of flights in and out of the airports
    this will then be used to calculate a column about % delayed 
    """
    airports_temp = pd.DataFrame(df_flights.groupby('origin_airport_id').count()['flights'])
    airports_temp.reset_index(inplace=True)
    airports_temp.rename(columns={'origin_airport_id':'airport_id', 'flights':'dep_flight_count'}, inplace=True)
    df_airports = df_airports.merge(airports_temp, on='airport_id')
    
    airports_temp = pd.DataFrame(df_flights.groupby('dest_airport_id').count()['flights'])
    airports_temp.reset_index(inplace=True)
    airports_temp.rename(columns={'dest_airport_id':'airport_id', 'flights':'arr_flight_count'}, inplace=True)
    df_airports = df_airports.merge(airports_temp, on='airport_id')    
    
    return df_airports

def create_delay_counts(df_flights, df_airports):
    """
    create columns counting number of flights lates for depart and arrival airports
    """
    airports_temp = pd.DataFrame(df_flights.groupby(by='origin_airport_id').agg('sum')['is_late'])
    airports_temp.reset_index(inplace=True)
    airports_temp.rename(columns={'origin_airport_id':'airport_id', 'is_late':'dep_late_count'}, inplace=True)
    df_airports = df_airports.merge(airports_temp, on='airport_id')

    airports_temp = pd.DataFrame(df_flights.groupby(by='dest_airport_id').agg('sum')['is_late'])
    airports_temp.reset_index(inplace=True)
    airports_temp.rename(columns={'dest_airport_id':'airport_id', 'is_late':'arr_late_count'}, inplace=True)
    df_airports = df_airports.merge(airports_temp, on='airport_id')

    return df_airports

def create_late_per(df_airports):
    """
    creates columns showing percent late for arrival and departing airports
    """
    df_airports['dep_late_perc'] = (df_airports['dep_late_count'] / df_airports['dep_flight_count']).round(3)
    df_airports['arr_late_perc'] = (df_airports['arr_late_count'] / df_airports['arr_flight_count']).round(3)

    return df_airports

def create_taxi(df_flights, df_airports):
    """
    add columns shows mean taxi time
    this will then be used to calculate a column about % delayed 
    """
    airports_temp = pd.DataFrame(df_flights.groupby('origin_airport_id').agg('mean')['taxi_out'])
    airports_temp.reset_index(inplace=True)
    airports_temp.rename(columns={'origin_airport_id':'airport_id', 'taxi_out':'dep_taxi'}, inplace=True)
    df_airports = df_airports.merge(airports_temp, on='airport_id')
    
    airports_temp = pd.DataFrame(df_flights.groupby('dest_airport_id').agg('mean')['taxi_in'])
    airports_temp.reset_index(inplace=True)
    airports_temp.rename(columns={'dest_airport_id':'airport_id', 'taxi_in':'arr_taxi'}, inplace=True)
    df_airports = df_airports.merge(airports_temp, on='airport_id')    
    
    return df_airports

    ## Tail Number Table ##
def create_tail_table(df_flights):
    """
    creates tabel with tail number stats from flights table
    """
    df_tail = df_flights[['tail_num', 'fl_date', 'is_late', 'arr_delay', 'arr_time', 'distance', 'carrier_delay', 
                          'late_aircraft_delay', 'weather_delay']].groupby(by='tail_num').agg({'fl_date':'count', 
                                                                                               'is_late':'sum', 
                                                                                               'arr_delay':['median', 'mean', 'std'],
                                                                                               'arr_time':['median', 'mean'],
                                                                                               'distance':'mean',
                                                                                               'carrier_delay':'count',
                                                                                               'late_aircraft_delay':'count',
                                                                                               'weather_delay':'count'})
    df_tail['late_perc'] = (df_tail['is_late']['sum'] / df_tail['fl_date']['count']).round(3)
    
    #flatterning the multi layered headers for easier use later
    flat_cols = list(df_tail.columns.map('_'.join))
    df_tail.columns = flat_cols

    return df_tail

    ## Carrier Table ##
def create_carrier_table(df_flights):
    """
    creates tabel with carrier stats from flights table
    """
    df_carrier = df_flights[['mkt_carrier', 'fl_date', 'is_late', 
                             'dep_delay', 'arr_delay', 'carrier_delay']].groupby(by='mkt_carrier').agg({'fl_date':'count',
                                                                                           'is_late':'sum',
                                                                                           'dep_delay': 'mean',
                                                                                           'arr_delay': 'mean',
                                                                                           'carrier_delay': 'mean'})
    df_carrier['late_perc'] = (df_carrier['is_late'] / df_carrier['fl_date']).round(3)
    df_carrier['carrier_delay'] = df_carrier['carrier_delay'].round(2)
    df_carrier['dep_delay'] = df_carrier['dep_delay'].round(2)
    df_carrier['arr_delay'] = df_carrier['arr_delay'].round(2)

    return df_carrier

    ## Hourly Table ##
def create_hourly_table(df_flights):
    """
    creates tabel with hourly stats from flights table
    """
    df_hours = df_flights[['dep_hour', 'fl_date', 'is_late', 
                             'dep_delay', 'arr_delay']].groupby(by='dep_hour').agg({'fl_date':'count',
                                                                                           'is_late':'sum',
                                                                                           'dep_delay': 'mean',
                                                                                           'arr_delay': 'mean'})
    df_hours['late_perc'] = (df_hours['is_late'] / df_hours['fl_date']).round(3)
    df_hours['dep_delay'] = df_hours['dep_delay'].round(2)
    df_hours['arr_delay'] = df_hours['arr_delay'].round(2)

    return df_hours

    ## Daily Table ##
def create_day_table(df_flights):
    """
    creates tabel with day stats from flights table
    """
    df_days = df_flights[['day', 'fl_date', 'is_late', 
                             'dep_delay', 'arr_delay']].groupby(by='day').agg({'fl_date':'count',
                                                                                           'is_late':'sum',
                                                                                           'dep_delay': 'mean',
                                                                                           'arr_delay': 'mean'})
    df_days['late_perc'] = (df_days['is_late'] / df_days['fl_date']).round(3)
    df_days['dep_delay'] = df_days['dep_delay'].round(2)
    df_days['arr_delay'] = df_days['arr_delay'].round(2)
    
    #bucket carrier category as orginal

    return df_days

###### Exporting Created Tables ######
def save_tables():
    airports.to_csv('../../data/Exported_Tables/stats_airports.csv',  index=False)
    tail.to_csv('../../data/Exported_Tables/stats_tail.csv')
    carrier.to_csv('../../data/Exported_Tables/stats_carrier.csv')
    hourly.to_csv('../../data/Exported_Tables/stats_hourly.csv')
    daily.to_csv('../../data/Exported_Tables/stats_daily.csv')


###### Loading Created Tables ######
def load_f_tables():
    airports = pd.read_csv('../../data/Exported_Tables/stats_airports.csv')
    tail = pd.read_csv('../../data/Exported_Tables/stats_tail.csv',index_col='tail_num')
    carrier = pd.read_csv('../../data/Exported_Tables/stats_carrier.csv', index_col='mkt_carrier')
    hourly = pd.read_csv('../../data/Exported_Tables/stats_hourly.csv', index_col='dep_hour')
    daily = pd.read_csv('../../data/Exported_Tables/stats_daily.csv', index_col='day')

    return airports, tail, carrier, hourly, daily

###### Mearging Features Onto Origional Table ######

def get_features_airport(df_flights, df_airports):
    """
    merges engineered features from the airport stats table onto the main dataframe as additional columns
    """
    airport_dep = df_airports[['airport_id', 'mean_d_delay', 'dep_late_perc', 'dep_taxi']].rename(columns={'mean_d_delay':'air_mean_d_delay', 'dep_late_perc':'air_dep_late_perc', 'dep_taxi':'air_dep_taxi'})
    airport_arr = df_airports[['airport_id', 'mean_arr_delay', 'arr_late_perc', 'arr_taxi']].rename(columns={'mean_arr_delay':'air_mean_arr_delay', 'arr_late_perc':'air_arr_late_perc', 'arr_taxi':'air_arr_taxi'})

    #merge each onto the main dataframe and rename

    df_temp = df_flights.merge(airport_dep, left_on='origin_airport_id', right_on='airport_id')
    df_temp = df_temp.merge(airport_arr, left_on='dest_airport_id', right_on='airport_id')
    df_temp.drop(['airport_id_x', 'airport_id_y'], axis = 1, inplace=True)

    return df_temp

def get_features_hourly(df_flights, df_hourly):
    """
    merges engineered features from the hourly stats table onto the main dataframe as additional columns
    """
    hourly_filter = df_hourly.reset_index()[['dep_hour', 'fl_date', 
                                             'arr_delay', 'late_perc']].rename(columns={'fl_date':'hour_count', 
                                                                                        'arr_delay':'hour_arr_delay',
                                                                                        'late_perc':'hour_late_perc'})
    df_temp = df_flights.merge(hourly_filter, left_on='dep_hour', right_on='dep_hour')
    #df_temp.drop(['airport_id_x', 'airport_id_y'], axis = 1, inplace=True)

    return df_temp

def get_features_tail(df_flights, df_tail):
    """
    merges engineered features from the Tail Num stats table onto the main dataframe as additional columns
    """
    tail_filter = df_tail.reset_index()[['tail_num', 'fl_date_count', 'arr_delay_mean', 'arr_delay_std', 'late_perc_']]
    tail_filter.rename(columns={'fl_date_count':'tail_count', 'arr_delay_mean':'tail_arr_delay','arr_delay_std':'tail_arr_delay_std' , 'late_perc_':'tail_late_perc'}, inplace=True)
    tail_filter['tail_arr_delay'] = tail_filter['tail_arr_delay'].round(2)
    tail_filter['tail_arr_delay_std'] = tail_filter['tail_arr_delay_std'].round(2)
    tail_filter.drop('tail_arr_delay_std', axis=1, inplace=True)

    df_output = df_flights.merge(tail_filter, left_on='tail_num', right_on='tail_num')

    return df_output

def get_features_carrier(df_flights, df_carrier):
    """
    merges engineered features from carries
    """
    carrier_filter = df_carrier.reset_index()[['mkt_carrier', 'fl_date', 'arr_delay', 
                                               'carrier_delay', 'late_perc']].rename(columns={'fl_date':'carrier_count', 
                                                                                              'arr_delay':'carrier_arr_delay',
                                                                                              'carrier_delay':'carrier_carrier_delay',
                                                                                              'late_perc':'carrier_late_perc'})
    temp = df_flights.merge(carrier_filter, left_on='mkt_carrier', right_on='mkt_carrier')

    return temp

def get_features_day(df_flights, df_daily):
    daily_filter = df_daily.reset_index()[['day', 'fl_date', 'dep_delay', 'arr_delay', 
                                            'late_perc']].rename(columns={'fl_date':'day_count', 
                                                                          'dep_delay':'day_dep_delay',
                                                                          'arr_delay':'day_arr_delay',
                                                                          'late_perc':'day_late_perc'})
    
    output = df_flights.merge(daily_filter, left_on='day', right_on='day')

    return output 


###### Drop Uneeded Columns ######
def drop_it_og(df):
    """
    drop unneeded columns to be used in the model
    """
    output = df.copy().drop(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier','mkt_carrier_fl_num', 
                'op_unique_carrier', 'tail_num', 'op_carrier_fl_num', 'origin_airport_id','origin', 'origin_city_name', 'dest_airport_id',
                'dest', 'dest_city_name', 'crs_dep_time', 'dep_time', 'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in', 
                'crs_arr_time', 'arr_time', 'cancelled', 'cancellation_code', 'diverted', 'dup', 'actual_elapsed_time', 'air_time', 'flights',
                'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay', 'first_dep_time', 'total_add_gtime', 
                'longest_add_gtime', 'no_name', 'year'], axis=1)
    return output

def save_final_features(df):
    """
    export csv final features table to specificed location
    """
    df.to_csv('../../data/Exported_Tables/final_features.csv')
    return 'Sucessfully exported final features'

## Process

In [7]:
#### LOAD & CLEAN ####
df = (
            load_data()
            .pipe(remove_rows)
            .pipe(create_datetime)
            .pipe(create_is_late)
            .pipe(create_speed)
            )
print('---- Basic Proprocessing Complete! ----')

#### TABLE CREATION ####
# create airport table
airports = create_table_airports()
airports = create_dep_delay(df, airports)
airports = create_arr_delay(df, airports)
airports = create_flight_counts(df, airports)
airports = create_delay_counts(df, airports)
airports = create_late_per(airports)
airports = create_taxi(df, airports)
print('---- Airport Table Complete! ----')

# create tail numbers table
tail = create_tail_table(df)
print('---- Tail Table Complete!    ----')
# create carrier table
carrier = create_carrier_table(df)
print('---- Carrier Table Complete! ----')
# create hourly table
hourly = create_hourly_table(df)
print('---- hourly Table Complete!  ----')
# create daily table
daily = create_day_table(df)
print('---- Daily Table Complete!   ----')
# Export all tables for later use #
save_tables()
print('---- Export Tables Complete! ----')

#### Load Created Tables #####
airports, tail, carrier, hourly, daily = load_f_tables()
print('---- Loading Complete!       ----')

#### MERGING TO DF ##### 
df_merged = df.copy()
df_merged = get_features_airport(df_merged, airports)
df_merged = get_features_hourly(df_merged, hourly)
df_merged = get_features_tail(df_merged, tail)
df_merged = get_features_carrier(df_merged, carrier)
df_merged = get_features_day(df_merged, daily)
print('---- Merge Complete!         ----')

#### DROP UNEEEDED ####
df_dropped = drop_it_og(df_merged)
print('---- Final Drop Complete!    ----')

#### DROP UNEEEDED ####
save_final_features(df_dropped)
print('---- Saving Complete!        ----')

/Users/josephluiz/miniconda3/envs/THIRD_LOVE/lib/python3.7/site-packages/ipykernel_launcher.py:3: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  This is separate from the ipykernel package so we can avoid doing imports until


---- Basic Proprocessing Complete! ----
---- Airport Table Complete! ----
---- Tail Table Complete!    ----
---- Carrier Table Complete! ----
---- hourly Table Complete!  ----
---- Daily Table Complete!   ----
---- Export Tables Complete! ----
---- Loading Complete!       ----
---- Merge Complete!         ----
---- Final Drop Complete!    ----
---- Saving Complete!        ----


# Scrap

In [8]:
df

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name,month,year,day,dep_hour,arr_hour,is_late,speed
0,2019-05-19,AS,AS,AS,66,AS,N619AS,66,10299,ANC,"Anchorage, AK",10926,CDV,"Cordova, AK",1525,1624.0,59.0,14.0,1638.0,1713.0,4.0,1610,1717.0,67.0,0,NaN,0,N,45,53.0,35.0,1,160,59.0,0.0,8.0,0.0,0.0,1532.0,17.0,17.0,NaN,5,2019,6,15.0,16.0,1,3.56
1,2019-05-19,AS,AS,AS,85,AS,N265AK,85,10397,ATL,"Atlanta, GA",14747,SEA,"Seattle, WA",625,622.0,-3.0,15.0,637.0,813.0,8.0,845,821.0,-24.0,0,NaN,0,N,320,299.0,276.0,1,2182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,2019,6,6.0,8.0,0,6.82
2,2019-05-19,AS,AS,AS,143,AS,N590AS,143,14057,PDX,"Portland, OR",10299,ANC,"Anchorage, AK",2040,2032.0,-8.0,16.0,2048.0,2249.0,4.0,2320,2253.0,-27.0,0,NaN,0,N,220,201.0,181.0,1,1542,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,2019,6,20.0,23.0,0,7.01
3,2019-05-19,AS,AS,AS,317,AS,N442AS,317,11618,EWR,"Newark, NJ",14771,SFO,"San Francisco, CA",1940,2148.0,128.0,119.0,2347.0,224.0,7.0,2305,231.0,206.0,0,NaN,0,N,385,463.0,337.0,1,2565,0.0,0.0,78.0,0.0,128.0,NaN,NaN,NaN,NaN,5,2019,6,19.0,23.0,1,6.66
4,2019-05-19,AS,AS,AS,350,AS,N495AS,350,14747,SEA,"Seattle, WA",12266,IAH,"Houston, TX",1200,1201.0,1.0,28.0,1229.0,1827.0,11.0,1820,1838.0,18.0,0,NaN,0,N,260,277.0,238.0,1,1874,0.0,0.0,18.0,0.0,0.0,NaN,NaN,NaN,NaN,5,2019,6,12.0,18.0,1,7.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637436,2019-05-19,UA,UA_CODESHARE,UA,3601,YX,N651RW,3601,12266,IAH,"Houston, TX",10693,BNA,"Nashville, TN",1620,1620.0,0.0,24.0,1644.0,1811.0,31.0,1817,1842.0,25.0,0,NaN,0,N,117,142.0,87.0,1,657,0.0,0.0,25.0,0.0,0.0,NaN,NaN,NaN,NaN,5,2019,6,16.0,18.0,1,5.62
637437,2019-05-19,UA,UA_CODESHARE,UA,3611,YX,N729YX,3611,13198,MCI,"Kansas City, MO",12266,IAH,"Houston, TX",1830,1907.0,37.0,15.0,1922.0,2102.0,7.0,2037,2109.0,32.0,0,NaN,0,N,127,122.0,100.0,1,643,0.0,0.0,0.0,0.0,32.0,NaN,NaN,NaN,NaN,5,2019,6,18.0,20.0,1,5.06
637438,2019-05-19,UA,UA_CODESHARE,UA,3615,YX,N650RW,3615,13930,ORD,"Chicago, IL",13851,OKC,"Oklahoma City, OK",1246,1319.0,33.0,28.0,1347.0,1549.0,7.0,1507,1556.0,49.0,0,NaN,0,N,141,157.0,122.0,1,693,33.0,0.0,16.0,0.0,0.0,NaN,NaN,NaN,NaN,5,2019,6,12.0,15.0,1,4.91
637439,2019-05-19,WN,WN,WN,2980,WN,N296WN,2980,10140,ABQ,"Albuquerque, NM",14107,PHX,"Phoenix, AZ",1420,1450.0,30.0,9.0,1459.0,1457.0,6.0,1435,1503.0,28.0,0,NaN,0,N,75,73.0,58.0,1,328,10.0,0.0,0.0,0.0,18.0,NaN,NaN,NaN,NaN,5,2019,6,14.0,14.0,1,4.37


In [10]:
df
df['mkt_carrier'] + "_" + df['origin'] + "_" + df['dest']

0         AS_ANC_CDV
1         AS_ATL_SEA
2         AS_PDX_ANC
3         AS_EWR_SFO
4         AS_SEA_IAH
             ...    
637436    UA_IAH_BNA
637437    UA_MCI_IAH
637438    UA_ORD_OKC
637439    WN_ABQ_PHX
637440    AS_JNU_YAK
Length: 623434, dtype: object